In [2]:
import geopandas as gpd

# Input GeoPackage file
input_gpkg = "data/SOIL_DATA_v4.gpkg"

# Read data from GPKG
gdf = gpd.read_file(input_gpkg)

# 
gdf['POINT_ID'] = gdf['POINT_ID'].astype('int64')
gdf.set_index(keys='POINT_ID', inplace=True)
gdf = gdf[gdf["HAS_LANDUSE_CHANGED"]==0.0]
gdf.drop(axis=1, labels=["HAS_LANDUSE_CHANGED", "LANDUSE_CHANGE_VALUE_2012", "LANDUSE_CHANGE_LABEL_2012", "LANDUSE_CHANGE_VALUE_2018", "LANDUSE_CHANGE_LABEL_2018"], inplace=True)

# Save as CSV
output_csv = "data/data.csv"
gdf.to_csv(output_csv)

print(f"File {output_csv} has been successfully created.")

File data/data.csv has been successfully created.


In [3]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 13220 entries, 26581768 to 64981672
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   SOC_2009            13220 non-null  float64 
 1   pH_2009             13220 non-null  float64 
 2   CLAY_percent        13220 non-null  int32   
 3   SILT_percent        13220 non-null  int32   
 4   SAND_percent        13220 non-null  int32   
 5   SOC_2018            13220 non-null  float64 
 6   pH_2018             13220 non-null  float64 
 7   SOIL_REGION         13220 non-null  float64 
 8   KG_CLASS            13220 non-null  object  
 9   tp_2009             13220 non-null  float64 
 10  tp_2018             13220 non-null  float64 
 11  tp_s_2009           13220 non-null  float64 
 12  tp_s_2018           13220 non-null  float64 
 13  tp_w_2009           13220 non-null  float64 
 14  tp_w_2018           13220 non-null  float64 
 15  pr_2009             132

In [4]:
soil_aggr = set()
for aggr in gdf["soil_region_aggr"].unique():
    soil_aggr = soil_aggr.union(aggr.split(sep="; "))

soil_aggr_cols = [f"soil_region_aggr_{type}" for type in soil_aggr]

soil_aggr

{'AB',
 'AC',
 'AN',
 'AR',
 'AT',
 'CH',
 'CL',
 'CM',
 'FL',
 'GL',
 'GY',
 'HS',
 'LP',
 'LV',
 'PH',
 'PL',
 'PZ',
 'RG',
 'SC',
 'UM',
 'VR'}

In [5]:
import numpy as np

gdf[soil_aggr_cols] = 0

@np.vectorize
def aggr_split(index: int):
    gdf.loc[index, [f"soil_region_aggr_{type}" for type in gdf.loc[index, "soil_region_aggr"].split(sep="; ")]] = 1

aggr_split(gdf.index.to_list())

gdf

,SOC_2009,pH_2009,CLAY_percent,SILT_percent,SAND_percent,SOC_2018,pH_2018,SOIL_REGION,KG_CLASS,tp_2009,...,soil_region_aggr_CM,soil_region_aggr_LV,soil_region_aggr_AN,soil_region_aggr_RG,soil_region_aggr_PH,soil_region_aggr_HS,soil_region_aggr_FL,soil_region_aggr_SC,soil_region_aggr_AC,soil_region_aggr_CH
POINT_ID,,,,,,,,,,,,,,,,,,,,,
26581768,56.9,7.40,25,29,46,28.2,7.47,254.0,Csa,16.634109,...,1,0,0,1,0,0,0,0,0,0
26581792,19.7,6.09,26,49,25,33.5,6.51,219.0,Csa,16.186918,...,0,0,0,0,0,0,0,0,0,0
26581954,14.5,8.21,34,37,29,13.7,7.58,218.0,Csb,16.592876,...,1,0,0,0,0,0,0,0,0,0
26601784,72.1,5.41,22,53,25,52.1,5.54,254.0,Csa,15.935137,...,1,0,0,1,0,0,0,0,0,0
26702014,24.1,5.96,16,16,67,17.9,5.81,218.0,Csb,15.057329,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64601638,19.8,8.43,33,39,27,22.0,8.18,258.0,BSh,21.128698,...,1,0,0,0,0,0,0,0,0,0
64621644,16.8,8.43,32,42,26,15.4,8.26,258.0,BSh,21.086849,...,1,0,0,0,0,0,0,0,0,0
64841666,19.5,8.42,32,19,49,8.2,7.61,258.0,BSh,20.138493,...,1,0,0,0,0,0,0,0,0,0


In [6]:
import pandas as pd

gdf["SOC_INCREASE"] = gdf["SOC_2018"] - gdf["SOC_2009"]
gdf = pd.concat([gdf.drop(labels=["KG_CLASS", "LANDUSE_LABEL_2012", "LANDUSE_LABEL_2018", "soil_region_aggr"], axis=1), pd.get_dummies(data=gdf[["KG_CLASS", "LANDUSE_LABEL_2012", "LANDUSE_LABEL_2018"]]) + 0], axis=1, ignore_index=False)

gdf.to_csv(output_csv)

gdf

,SOC_2009,pH_2009,CLAY_percent,SILT_percent,SAND_percent,SOC_2018,pH_2018,SOIL_REGION,tp_2009,tp_2018,...,LANDUSE_LABEL_2012_Heterogeneous agric. areas,LANDUSE_LABEL_2012_Inland wetlands,LANDUSE_LABEL_2012_Permanent crops,LANDUSE_LABEL_2012_Shrubs,LANDUSE_LABEL_2018_Arable land,LANDUSE_LABEL_2018_Forests,LANDUSE_LABEL_2018_Heterogeneous agric. areas,LANDUSE_LABEL_2018_Inland wetlands,LANDUSE_LABEL_2018_Permanent crops,LANDUSE_LABEL_2018_Shrubs
POINT_ID,,,,,,,,,,,,,,,,,,,,,
26581768,56.9,7.40,25,29,46,28.2,7.47,254.0,16.634109,15.809589,...,1,0,0,0,0,0,1,0,0,0
26581792,19.7,6.09,26,49,25,33.5,6.51,219.0,16.186918,15.492535,...,0,0,0,1,0,0,0,0,0,1
26581954,14.5,8.21,34,37,29,13.7,7.58,218.0,16.592876,15.900685,...,0,0,0,1,0,0,0,0,0,1
26601784,72.1,5.41,22,53,25,52.1,5.54,254.0,15.935137,15.296027,...,0,0,0,0,0,1,0,0,0,0
26702014,24.1,5.96,16,16,67,17.9,5.81,218.0,15.057329,14.625891,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64601638,19.8,8.43,33,39,27,22.0,8.18,258.0,21.128698,22.136232,...,0,0,0,0,1,0,0,0,0,0
64621644,16.8,8.43,32,42,26,15.4,8.26,258.0,21.086849,22.074657,...,0,0,0,0,1,0,0,0,0,0
64841666,19.5,8.42,32,19,49,8.2,7.61,258.0,20.138493,21.468767,...,1,0,0,0,0,0,1,0,0,0


In [7]:
features = gdf.drop(labels=["SOC_2009", "SOC_2018", "SOIL_REGION", "SOC_INCREASE", "geometry"], axis=1)
results = gdf["SOC_INCREASE"]

features.to_csv("data/features.csv")
results.to_csv("data/results.csv")

In [8]:
results.max()

np.float64(562.8)

In [9]:
results.min()

np.float64(-507.2)